# COVID-19 French Maps
Guillaume Rozier, 2020

In [116]:
"""

LICENSE MIT
2020
Guillaume Rozier
Website : http://www.guillaumerozier.fr
Mail : guillaume.rozier@telecomnancy.net

README:s
This file contains script that generate France maps and GIFs. 
Single images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.
I'm currently cleaning this file, please ask me is something is not clear enough!
Requirements: please see the imports below (use pip3 to install them).

"""

"\n\nLICENSE MIT\n2020\nGuillaume Rozier\nWebsite : http://www.guillaumerozier.fr\nMail : guillaume.rozier@telecomnancy.net\n\nREADME:s\nThis file contains script that generate France maps and GIFs. \nSingle images are exported to folders in 'charts/image/france'. GIFs are exported to 'charts/image/france'.\nI'm currently cleaning this file, please ask me is something is not clear enough!\nRequirements: please see the imports below (use pip3 to install them).\n\n"

In [117]:
import france_data_management as data
import pandas as pd
from tqdm import tqdm
import json
import plotly.express as px
from datetime import datetime
import imageio
import multiprocessing
import locale
locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')

'fr_FR.UTF-8'

## Data import

In [118]:
# Import data from Santé publique France
df, df_confirmed, dates, _, _, df_deconf, df_sursaud = data.import_data()


 75%|███████▌  | 3/4 [00:00<00:00, 17.54it/s]

In [119]:
# Download and import data from INSEE
dict_insee = pd.read_excel('data/france/deces_quotidiens_departement.xlsx', header=[3], index_col=None, sheet_name=None, usecols='A:H', nrows=44)
dict_insee.pop('France')
dict_insee.pop('Documentation')

for key in dict_insee:
    dict_insee[key]["dep"] = [key for i in range(len(dict_insee[key]))]
    
df_insee = pd.concat(dict_insee)
df_insee = df_insee.rename(columns={"Ensemble des communes": "dc20", "Ensemble des communes.1": "dc19", "Ensemble des communes.2": "dc18", "Date d'événement": "jour"})
df_insee = df_insee.drop(columns=['Communes à envoi dématérialisé au 1er avril 2020 (1)', 'Communes à envoi dématérialisé au 1er avril 2020 (1)', 'Communes à envoi dématérialisé au 1er avril 2020 (1)', 'Unnamed: 7'])
df_insee["moy1819"] = (df_insee["dc19"] + df_insee["dc20"])/2
df_insee["surmortalite20"] = (df_insee["dc20"] - df_insee["moy1819"])/df_insee["moy1819"]*100
df_insee['jour'] = pd.to_datetime(df_insee['jour'])
df_insee['jour'] = df_insee['jour'].dt.strftime('%Y-%m-%d')

dates_insee = list(dict.fromkeys(list(df_insee.dropna()['jour'].values))) 

In [120]:
df_insee_france = df_insee.groupby('jour').sum().reset_index()
df_insee_france["surmortalite20"] = (df_insee_france["dc20"] - df_insee_france["moy1819"])/df_insee_france["moy1819"]

In [121]:
df_insee_france[df_insee_france["jour"] == "2020-04-06"]


,jour,dc20,Communes à envoi dématérialisé au 1er avril 2020 (1).1,dc19,Communes à envoi dématérialisé au 1er avril 2020 (1).2,dc18,moy1819,surmortalite20
36,2020-04-06,77763,58141,63686,64591,71003,70724.5,0.09952


<br>
<br>

## Function definition

In [122]:
with open('data/france/dep.geojson') as response:
    depa = json.load(response)

In [123]:

    
def map_gif(dates, imgs_folder, df, type_ppl, legend_title, min_scale, max_scale, colorscale, subtitle):
    i=1
    for date in tqdm(dates):
        if max_scale == -1:
            max_scale = df[type_ppl].max()
        df_map = pd.melt(df, id_vars=['jour','dep'], value_vars=[type_ppl])
        df_map = df_map[df_map["jour"] == date]

        data={}
        for dep in df_map["dep"].values:
            data[dep] = df_map[df_map["dep"] == dep]["value"]

        fig = px.choropleth(geojson=depa, 
                            locations=df_map['dep'], 
                            color=df_map['value'],
                            color_continuous_scale = colorscale,
                            range_color=(min_scale, max_scale),
                            featureidkey="properties.code",
                            scope='europe',
                            labels={'color':legend_title}
                                  )
        date_title = datetime.strptime(date, '%Y-%m-%d').strftime('%d %B')
        
        fig.update_geos(fitbounds="locations", visible=False)
        
        var_hab = 'pour 100k. hab.'
        pourcent = ''
        
        val_mean = round(df_map['value'].mean(), 1)
        
        n = len(dates)
        progressbar = i * '█' + (n-i) * '░'
        i += 1
        
        if type_ppl == 'surmortalite20':
            var_hab = ''
            pourcent = " %"
            if val_mean < 0:
                val_mean = "– " + str(abs(val_mean))
            else:
                val_mean = "+ " + str(val_mean)
                
        val_mean = str(val_mean).replace(".", ",")
        
        fig.update_layout(
            margin={"r":0,"t":0,"l":0,"b":0},
            title={
            'text': "{}".format(date_title),
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
            titlefont = dict(
            size=30),
            annotations = [
                dict(
                    x=0.54,
                    y=0.08,
                    xref='paper',
                    yref='paper',
                    xanchor = 'center',
                    text='Source : INSEE. Auteur : @guillaumerozier.',
                    showarrow = False
                ),
                dict(
                    x=0.54,
                    y=0.03,
                    xref = 'paper',
                    yref = 'paper',
                    text = progressbar,
                    xanchor = 'center',
                    showarrow = False,
                    font=dict(
                        size=14
                            )
                ),
                dict(
                    x=0.07,
                    y=0.47,
                    xref='paper',
                    yref='paper',
                    xanchor='left',
                    text='Moyenne France',
                    showarrow = False,
                    font=dict(
                        size=14
                            )
                ),
                dict(
                    x=0.07,
                    y=0.50,
                    xref='paper',
                    yref='paper',
                    xanchor='left',
                    text='{}{}'.format(val_mean, pourcent),
                    showarrow = False,
                    font=dict(
                        size=25
                            )
                ),
                
                dict(
                    x=0.07,
                    y=0.45,
                    xref='paper',
                    yref='paper',
                    xanchor='left',
                    text = var_hab,
                    showarrow = False,
                    font=dict(
                        size=14
                            )
                ),
                dict(
                    x=0.55,
                    y=0.9,
                    xref='paper',
                    yref='paper',
                    text=subtitle,
                    showarrow = False,
                    font=dict(
                        size=20
                            )
                )]
             ) 
        
        fig.update_geos(
            #center=dict(lon=-30, lat=-30),
            projection_rotation=dict(lon=12, lat=30, roll=8),
            #lataxis_range=[-50,20], lonaxis_range=[0, 200]
        )
        fig.write_image(imgs_folder.format(date), scale=1, width=900, height=700)


def build_gif(file_gif, imgs_folder, dates):
    i=0
    with imageio.get_writer(file_gif, mode='I', duration=0.2) as writer: 
        for date in tqdm(dates):
            image = imageio.imread(imgs_folder.format(date))
            writer.append_data(image)
            i+=1
            if i==len(dates):
                for k in range(4):
                    writer.append_data(image)

In [124]:

    
def build_map(data_df, img_folder, legend_title="legend_title", title="title"):
    dates_deconf = list(dict.fromkeys(list(data_df['extract_date'].values))) 
    date = dates_deconf[-1]
    
    data_df = data_df[data_df["extract_date"] == date]

    fig = px.choropleth(geojson = depa, 
                        locations = data_df['departement'], 
                        featureidkey="properties.code",
                        color = data_df['indic_synthese'],
                        scope='europe',
                        #labels={'indic_synthese':"Couleur"},
                        #color_discrete_sequence = ["green", "orange", "red"],
                        color_discrete_map = {"vert":"green", "orange":"orange", "rouge":"red"}
                        #category_orders = {"indic_synthese" :["vert", "orange", "rouge"]}
                              )
    date_title = datetime.strptime(dates_deconf[-1], '%Y-%m-%d').strftime('%d %B')

    fig.update_geos(fitbounds="locations", visible=False)

    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        title={
            'text': title,
            'y':0.98,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        
        titlefont = dict(
            size=30),
        
        annotations = [
            dict(
                x=0.54,
                y=0.03,
                xref='paper',
                yref='paper',
                xanchor = 'center',
                text='Source : Ministère de la Santé. Auteur : @guillaumerozier.',
                showarrow = False
            ),

            dict(
                x=0.55,
                y=0.94,
                xref='paper',
                yref='paper',
                text= "Mis à jour le {}".format(date_title),
                showarrow = False,
                font=dict(
                    size=20
                        )
            )]
         ) 

    fig.update_geos(
        #center=dict(lon=-30, lat=-30),
        projection_rotation=dict(lon=12, lat=30, roll=8),
        #lataxis_range=[-50,20], lonaxis_range=[0, 200]
    )
    #fig.show()
    if date == dates_deconf[-1]:
        fig.write_image(img_folder.format("latest"), scale=2, width=1200, height=800)
    fig.write_image(img_folder.format(date), scale=2, width=1200, height=800)

In [125]:
build_map(df_deconf, img_folder="images/charts/france/deconf_synthese/{}.png", title="Départements déconfinés le 11/05")


In [126]:
def build_map_indic1(data_df, img_folder, legend_title="legend_title", title="title"):
    dates_deconf = list(dict.fromkeys(list(data_df['date_de_passage'].values))) 
    date = dates_deconf[-1]
    
    data_df = data_df[data_df["date_de_passage"] == date]

    fig = px.choropleth(geojson = depa, 
                        locations = data_df['dep'], 
                        featureidkey="properties.code",
                        color = data_df['taux_corona'],
                        scope='europe',
                        range_color=(0, 0.1)
                        #labels={'indic_synthese':"Couleur"},
                        #color_discrete_sequence = ["green", "orange", "red"],
                        #color_discrete_map = {"vert":"green", "orange":"orange", "rouge":"red"}
                        #category_orders = {"indic_synthese" :["vert", "orange", "rouge"]}
                              )
    date_title = datetime.strptime(dates_deconf[-1], '%Y-%m-%d').strftime('%d %B')

    fig.update_geos(fitbounds="locations", visible=False)

    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0},
        title={
            'text': title,
            'y':0.98,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'},
        
        titlefont = dict(
            size=30),
        
        annotations = [
            dict(
                x=0.54,
                y=0.03,
                xref='paper',
                yref='paper',
                xanchor = 'center',
                text='Source : Ministère de la Santé. Auteur : @guillaumerozier.',
                showarrow = False
            ),

            dict(
                x=0.55,
                y=0.94,
                xref='paper',
                yref='paper',
                text= "Mis à jour le {}".format(date_title),
                showarrow = False,
                font=dict(
                    size=20
                        )
            )]
         ) 

    fig.update_geos(
        #center=dict(lon=-30, lat=-30),
        projection_rotation=dict(lon=12, lat=30, roll=8),
        #lataxis_range=[-50,20], lonaxis_range=[0, 200]
    )
    #fig.show()
    if date == dates_deconf[-1]:
        fig.write_image(img_folder.format("latest"), scale=2, width=1200, height=800)
    fig.write_image(img_folder.format(date), scale=2, width=1200, height=800)
    


In [127]:
df_sursaud = df_sursaud[df_sursaud["sursaud_cl_age_corona"] == "0"]
df_sursaud_gb = df_sursaud.groupby(["dep", "date_de_passage"]).rolling(window=7).sum().reset_index()

df_sursaud["taux_corona"] = df_sursaud["nbre_pass_corona"]/df_sursaud["nbre_pass_tot"]
df_sursaud["taux_corona"] = df_sursaud["taux_corona"].rolling(window=7).sum()

df_sursaud_gb["taux_corona"] = df_sursaud_gb["nbre_pass_corona"]/df_sursaud_gb["nbre_pass_tot"]

build_map_indic1(df_sursaud_gb, img_folder="images/charts/france/deconf_indic1/{}.png", title="Indic 1")

In [128]:
#df_sursaud_gb = df_sursaud.groupby(["dep"]).rolling(window=7, on="date_de_passage").mean().reset_index()
#df_sursaud_gb["date_de_passage"] = df_sursaud["date_de_passage"].values
#df_sursaud_gb[df_sursaud_gb["dep"]=="01"]
#df_sursaud_gb


In [129]:
df_sursaud_test = df_sursaud.groupby("dep")["nbre_pass_tot"].rolling(window=2).sum().reset_index()
#df_sursaud_test["date_de_passage"] = df_sursaud["date_de_passage"].values
df_sursaud_test

,dep,level_1,nbre_pass_tot
0,01,0,NaN
1,01,606,667.0
2,01,1212,676.0
3,01,1818,687.0
4,01,2424,660.0
...,...,...,...
6863,976,38778,218.0
6864,976,39384,263.0
6865,976,39990,253.0
6866,976,40596,225.0


In [130]:
df_sursaud.sort_values(by=["dep", "date_de_passage"])

,dep,date_de_passage,sursaud_cl_age_corona,nbre_pass_corona,nbre_pass_tot,nbre_hospit_corona,nbre_pass_corona_h,nbre_pass_corona_f,nbre_pass_tot_h,nbre_pass_tot_f,nbre_hospit_corona_h,nbre_hospit_corona_f,nbre_acte_corona,nbre_acte_tot,nbre_acte_corona_h,nbre_acte_corona_f,nbre_acte_tot_h,nbre_acte_tot_f,taux_corona
0,01,2020-02-24,0,0.0,357.0,0.0,0.0,0.0,202.0,155.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
606,01,2020-02-25,0,0.0,310.0,0.0,0.0,0.0,177.0,133.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1212,01,2020-02-26,0,0.0,366.0,0.0,0.0,0.0,193.0,173.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1818,01,2020-02-27,0,0.0,321.0,0.0,0.0,0.0,178.0,143.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2424,01,2020-02-28,0,0.0,339.0,0.0,0.0,0.0,166.0,173.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38778,976,2020-04-27,0,25.0,130.0,4.0,10.0,15.0,71.0,59.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39384,976,2020-04-28,0,23.0,133.0,1.0,7.0,16.0,74.0,59.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39990,976,2020-04-29,0,36.0,120.0,4.0,21.0,15.0,65.0,55.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40596,976,2020-04-30,0,14.0,105.0,5.0,6.0,8.0,59.0,46.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<br>

<br>

<br>

<br>

## Function calls

In [131]:
def dep_map():
    # GIF carte nb réanimations par habitant
    imgs_folder = "images/charts/france/dep-map-img/{}.png"
    sub = 'Nombre de <b>personnes en réanimation</b> <br>par habitant de chaque département.'
    map_gif(dates, imgs_folder, df = df, type_ppl = "rea_deppop", legend_title="réan./100k hab", min_scale = 0, max_scale=-1, colorscale ="Reds", subtitle=sub)
    build_gif(file_gif = "images/charts/france/dep-map.gif", imgs_folder = "images/charts/france/dep-map-img/{}.png", dates=dates)

In [132]:
def dep_map_dc_cum():
    # GIF carte décès cumulés par habitant
    imgs_folder = "images/charts/france/dep-map-img-dc-cum/{}.png"
    sub = 'Nombre de <b>décès cumulés</b> <br>par habitant de chaque département.'
    map_gif(dates[1:], imgs_folder, df = df, type_ppl = "dc_deppop", legend_title="décès/100k hab", min_scale = 0, max_scale=-1, colorscale ="Reds", subtitle=sub)
    build_gif(file_gif = "images/charts/france/dep-map-dc-cum.gif", imgs_folder = "images/charts/france/dep-map-img-dc-cum/{}.png", dates=dates[1:])

In [133]:
def dep_map_dc_journ():
    # GIF carte décès quotidiens 
    imgs_folder = "images/charts/france/dep-map-img-dc-journ/{}.png"
    sub = 'Nombre de <b>décès quotidien</b> <br>par habitant de chaque département.'
    map_gif(dates[1:], imgs_folder, df = df, type_ppl = "dc_new_deppop", legend_title="décès/100k hab", min_scale = 0, max_scale=-1, colorscale ="Reds", subtitle=sub)
    build_gif(file_gif = "images/charts/france/dep-map-dc-journ.gif", imgs_folder = "images/charts/france/dep-map-img-dc-journ/{}.png", dates=dates[1:])

In [134]:
dep_map()
dep_map_dc_cum()
dep_map_dc_journ()



  0%|          | 0/46 [00:00<?, ?it/s]

  2%|▏         | 1/46 [00:05<03:53,  5.18s/it]

  4%|▍         | 2/46 [00:11<03:56,  5.38s/it]

  7%|▋         | 3/46 [00:16<03:56,  5.51s/it]

  9%|▊         | 4/46 [00:22<03:47,  5.42s/it]

 11%|█         | 5/46 [00:27<03:46,  5.53s/it]

 13%|█▎        | 6/46 [00:34<03:57,  5.93s/it]

 15%|█▌        | 7/46 [00:41<04:02,  6.23s/it]

 17%|█▋        | 8/46 [00:46<03:46,  5.97s/it]

 20%|█▉        | 9/46 [00:52<03:35,  5.83s/it]

 22%|██▏       | 10/46 [00:58<03:30,  5.83s/it]

 24%|██▍       | 11/46 [01:04<03:23,  5.81s/it]

 26%|██▌       | 12/46 [01:09<03:17,  5.82s/it]

 28%|██▊       | 13/46 [01:15<03:14,  5.89s/it]

 30%|███       | 14/46 [01:22<03:09,  5.93s/it]

 33%|███▎      | 15/46 [01:27<03:01,  5.85s/it]

 35%|███▍      | 16/46 [01:33<02:52,  5.76s/it]

 37%|███▋      | 17/46 [01:38<02:46,  5.74s/it]

 39%|███▉      | 18/46 [01:44<02:36,  5.58s/it]

 41%|████▏     | 19/46 [01:49<02:27,  5.46s/it]

 43%|████▎     | 20/46 [01:54<02:20,

 62%|██████▏   | 28/45 [00:02<00:01,  9.15it/s]

 64%|██████▍   | 29/45 [00:02<00:01,  9.32it/s]

 67%|██████▋   | 30/45 [00:03<00:01,  9.33it/s]

 69%|██████▉   | 31/45 [00:03<00:01,  9.25it/s]

 71%|███████   | 32/45 [00:03<00:01,  9.14it/s]

 76%|███████▌  | 34/45 [00:03<00:01,  9.27it/s]

 78%|███████▊  | 35/45 [00:03<00:01,  8.47it/s]

 80%|████████  | 36/45 [00:03<00:01,  7.65it/s]

 82%|████████▏ | 37/45 [00:03<00:01,  7.65it/s]

 84%|████████▍ | 38/45 [00:04<00:00,  7.89it/s]

 87%|████████▋ | 39/45 [00:04<00:00,  8.23it/s]

 89%|████████▉ | 40/45 [00:04<00:00,  8.54it/s]

 93%|█████████▎| 42/45 [00:04<00:00,  8.88it/s]

 96%|█████████▌| 43/45 [00:04<00:00,  8.82it/s]

100%|██████████| 45/45 [00:05<00:00,  8.76it/s]


  0%|          | 0/45 [00:00<?, ?it/s]

  2%|▏         | 1/45 [00:05<04:00,  5.47s/it]

  4%|▍         | 2/45 [00:11<03:57,  5.53s/it]

  7%|▋         | 3/45 [00:16<03:53,  5.56s/it]

  9%|▉         | 4/45 [00:22<03:45,  5.51s/it]

 11%|█         | 5/45 [00:27<03:

In [135]:
"""def bestfunction(unefonc):
    unefonc()
    time.sleep(5)"""
    
"""pool = multiprocessing.Pool(processes=4) 
pool.apply_async(bestfunction, [dep_map(), dep_map_dc_cum(), dep_map_dc_journ()])"""

"""processes = []
for i in (dep_map, dep_map_dc_cum, dep_map_dc_journ):
    p = multiprocessing.Process(target = bestfunction, args=(i,))
    processes.append(p)
    p.start()

for process in processes:
    process.join()
"""

"""p = multiprocessing.Process(target=bestfunction, args=(dep_map(), dep_map_dc_cum(), dep_map_dc_journ()))
p.start()
p.join()"""

'p = multiprocessing.Process(target=bestfunction, args=(dep_map(), dep_map_dc_cum(), dep_map_dc_journ()))\np.start()\np.join()'

In [136]:
"""
# INSEE
# GIF mortalité par rapport à 2018 et 2019
imgs_folder = "images/charts/france/dep-map-surmortalite-img/{}.png"
ppl = "surmortalite20"
sub = 'Comparaison de la <b>mortalité journalière</b> entre 2020 <br>et les deux années précédentes.'
map_gif(dates_insee, imgs_folder, df = df_insee.dropna(), type_ppl = ppl, legend_title="Sur-mortalité (%)", min_scale=-50, max_scale=50, colorscale = ["green", "white", "red"], subtitle = sub)
build_gif(file_gif = "images/charts/france/dep-map-surmortalite.gif", imgs_folder = imgs_folder, dates=dates_insee)"""

'\n# INSEE\n# GIF mortalité par rapport à 2018 et 2019\nimgs_folder = "images/charts/france/dep-map-surmortalite-img/{}.png"\nppl = "surmortalite20"\nsub = \'Comparaison de la <b>mortalité journalière</b> entre 2020 <br>et les deux années précédentes.\'\nmap_gif(dates_insee, imgs_folder, df = df_insee.dropna(), type_ppl = ppl, legend_title="Sur-mortalité (%)", min_scale=-50, max_scale=50, colorscale = ["green", "white", "red"], subtitle = sub)\nbuild_gif(file_gif = "images/charts/france/dep-map-surmortalite.gif", imgs_folder = imgs_folder, dates=dates_insee)'

In [137]:
"""# Line chart évolution de la mortalité

import plotly.graph_objects as go
import plotly
fig = go.Figure()

fig.add_trace(go.Scatter(
    x = df_insee_france["jour"],
    y = df_insee_france["surmortalite20"],
    name = "Bilan autre hosp",
    marker_color='black',
    mode="lines+markers",
    opacity=1
))


# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    legend_orientation="v",
    barmode='relative',
    title={
                'text': "Variation de la <b>mortalité en mars 2020</b> par rapport à 2018 et 2019",
                'y':0.95,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'},
                titlefont = dict(
                size=20),
    xaxis=dict(
        title='',
        tickformat='%d/%m'),
    yaxis_title="Surmortalité (%)",
    
    annotations = [
                dict(
                    x=0,
                    y=1.05,
                    xref='paper',
                    yref='paper',
                    text='Date : {}. Source : INSEE et CSSE. Auteur : @guillaumerozier (Twitter).'.format(datetime.strptime(dates[-1], '%Y-%m-%d').strftime('%d %B %Y')),                    showarrow = False
                )]
                 )

fig.update_layout(
    yaxis = go.layout.YAxis(
        tickformat = '%'
    ),
    annotations = [
                dict(
                    x=0.5,
                    y=1.05,
                    xref='paper',
                    yref='paper',
                    xanchor='center',
                    text='',
                    showarrow = False
                )]
                 )

name_fig = "insee_surmortalite"
fig.write_image("images/charts/france/{}.png".format(name_fig), scale=2, width=1200, height=800)
plotly.offline.plot(fig, filename = 'images/html_exports/france/{}.html'.format(name_fig), auto_open=False)
print("> " + name_fig)

fig.show()"""

'# Line chart évolution de la mortalité\n\nimport plotly.graph_objects as go\nimport plotly\nfig = go.Figure()\n\nfig.add_trace(go.Scatter(\n    x = df_insee_france["jour"],\n    y = df_insee_france["surmortalite20"],\n    name = "Bilan autre hosp",\n    marker_color=\'black\',\n    mode="lines+markers",\n    opacity=1\n))\n\n\n# Here we modify the tickangle of the xaxis, resulting in rotated labels.\nfig.update_layout(\n    legend_orientation="v",\n    barmode=\'relative\',\n    title={\n                \'text\': "Variation de la <b>mortalité en mars 2020</b> par rapport à 2018 et 2019",\n                \'y\':0.95,\n                \'x\':0.5,\n                \'xanchor\': \'center\',\n                \'yanchor\': \'top\'},\n                titlefont = dict(\n                size=20),\n    xaxis=dict(\n        title=\'\',\n        tickformat=\'%d/%m\'),\n    yaxis_title="Surmortalité (%)",\n    \n    annotations = [\n                dict(\n                    x=0,\n                   